In [1]:
with open("out.tf", "r") as f:
    data = f.readlines()

In [2]:
EVENT, TIME, FROM_NODE, TO_NODE, PACKET_TYPE, PACKET_SIZE, FLAGS, FID, SOURCE_ADDRESS, DST_ADDRESS, SEQ_NUM, PKT_ID = [i for i in range(12)]

In [3]:
def filterPackets(data, condition):
    result = []
    for line in data:
        items = line.split()
        conditionSatisfied = True
        for key, value in condition.items():
            if items[key] != value:
                conditionSatisfied = False
                break
        if conditionSatisfied:
            result.append(items)
    return result

In [4]:
def getTotalPacketsSize(connection):
    return sum([int(packets[PACKET_SIZE]) for packets in connection])

In [5]:
tcp0Condition = {TO_NODE: '5', EVENT: 'r'}
tcp1Condition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'tcp'}
tcp2Condition = {TO_NODE: '7', EVENT: 'r'}
udpCondition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'cbr'}
tcp0PacketSum = getTotalPacketsSize(filterPackets(data, tcp0Condition))
tcp1PacketSum = getTotalPacketsSize(filterPackets(data, tcp1Condition))
tcp2PacketSum = getTotalPacketsSize(filterPackets(data, tcp2Condition))
udpPacketSum = getTotalPacketsSize(filterPackets(data, udpCondition))
print("Average Throughput of TCP0:", tcp0PacketSum / 10)
print("Average Throughput of TCP1:", tcp1PacketSum / 10)
print("Average Throughput of TCP2:", tcp2PacketSum / 10)
print("Average Throughput of UDP:", udpPacketSum / 1)

Average Throughput of TCP0: 106292.0
Average Throughput of TCP1: 37132.0
Average Throughput of TCP2: 34116.0
Average Throughput of UDP: 128000.0


In [6]:
def getAverageDelay(data, startCondition, endCondition):
    firstSentPackets = filterPackets(data, startCondition)
    startTime = {}
    for item in firstSentPackets:
        startTime[item[PKT_ID]] = float(item[TIME])
    lastReceivedPackets = filterPackets(data, endCondition)
    totalDelay = 0
    for item in lastReceivedPackets:
        totalDelay += float(item[TIME]) - startTime[item[PKT_ID]]
    return totalDelay / len(lastReceivedPackets)

In [7]:
print("Average Delay of TCP0:", getAverageDelay(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'}))
print("Average Delay of TCP1:", getAverageDelay(data, {FROM_NODE: '1', TO_NODE: '3', EVENT: '+', PACKET_TYPE: 'tcp'}, {FROM_NODE: '4', TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'tcp'}))
print("Average Delay of TCP2:", getAverageDelay(data, {FROM_NODE: '2', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '7', EVENT: 'r'}))
print("Average Delay of UDP:", getAverageDelay(data, {FROM_NODE: '1', TO_NODE: '3', EVENT: '+', PACKET_TYPE: 'cbr'}, {FROM_NODE: '4', TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'cbr'}))

Average Delay of TCP0: 0.21389742521994065
Average Delay of TCP1: 0.23257158938547495
Average Delay of TCP2: 0.18002489057750753
Average Delay of UDP: 0.29460734375
